<a href="https://colab.research.google.com/github/sarwarbeing-ai/sarai/blob/master/Chaii_Hindi_Tamil_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # numpy library
import pandas as pd # pandas library

In [4]:
# installing other library
!pip install tensorflow

In [5]:
import tensorflow as tf

In [6]:
tf.__version__

'2.6.0'

In [8]:
# install transformer
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-2nvk8bvq
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-2nvk8bvq
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 50 kB 3.8 MB/s 
     |████████████████████████████████| 636 kB 11.5 MB/s 
     |████████████████████████████████| 895 kB 29.5 MB/s 
     |████████████████████████████████| 3.3 MB 41.1 MB/s 
  Created wheel for transformers: filename=transformers-4.11.0.dev0-py3-none-any.whl size=2761732 sha256=8acf96cecd0488677b4f7376c8b6d10c93a88a1e77afc7b588eff883912e61a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-n71h0mip/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uni

In [9]:
import transformers as tns

In [10]:
tns.__version__

'4.11.0.dev0'

In [15]:
# install dataset
!pip install datasets

     |████████████████████████████████| 264 kB 6.9 MB/s 
     |████████████████████████████████| 119 kB 75.8 MB/s 
     |████████████████████████████████| 243 kB 68.4 MB/s 


In [31]:
import datasets as d
from sklearn.model_selection import KFold

In [17]:
d.__version__

'1.11.0'

In [32]:
# load  data
train_df=pd.read_csv('./train.csv')
test_df=pd.read_csv('./test.csv')

In [25]:
train_df.shape

(1114, 6)

In [33]:
test_df.shape

(5, 4)

In [28]:
train_df.head(10)

,id,context,question,answer_text,answer_start,language
0,903deec17,ஒரு சாதாரண வளர்ந்த மனிதனுடைய எலும்புக்கூடு பின...,மனித உடலில் எத்தனை எலும்புகள் உள்ளன?,206,53,tamil
1,d9841668c,காளிதாசன் (தேவநாகரி: कालिदास) சமஸ்கிருத இலக்கி...,காளிதாசன் எங்கு பிறந்தார்?,காசுமீரில்,2358,tamil
2,29d154b56,சர் அலெக்ஸாண்டர் ஃபிளெமிங் (Sir Alexander Flem...,பென்சிலின் கண்டுபிடித்தவர் யார்?,சர் அலெக்ஸாண்டர் ஃபிளெமிங்,0,tamil
3,41660850a,"குழந்தையின் அழுகையை நிறுத்தவும், தூங்க வைக்கவ...",தமிழ்நாட்டில் குழந்தைகளை தூங்க வைக்க பாடும் பா...,தாலாட்டு,68,tamil
4,b29c82c22,சூரியக் குடும்பம் \nசூரியக் குடும்பம் (Solar S...,பூமியின் அருகில் உள்ள விண்மீன் எது?,சூரியனும்,585,tamil
5,d419db018,மின்னணுவியல் (Electronics) மின்னணுக்கள் அல்லது...,திரிதடையங்களைப் பயன்படுத்திய முதல் நிறுவனம் எது?,IBM,4171,tamil
6,c29e29ab6,அலெக்ஸாண்டர் கிரகாம் பெல் ( Alexander Graham B...,தொலைபேசியைக் கண்டுபிடித்தவர் யார்?,அலெக்ஸாண்டர் கிரகாம் பெல்,0,tamil
7,2a5ba78e2,முதல் உலகப்போர் என்பது உலகம் தழுவிய அளவில் இடம...,முதலாம் உலகப்போர் எப்பொழுது துவங்கியது?,1914ம்,1070,tamil
8,10ff95f4c,இந்தியாவின் தேசிய மனித உரிமை ஆணையம் ஒரு தன்னா...,இந்தியாவில் மனித உரிமை ஆணையம் எப்போது நிறுவப்ப...,"அக்டோபர் 12, 1993",90,tamil
9,7a6e807d7,"நெல்சன் மண்டேலா (Nelson Rolihlahla Mandela, 18...",நெல்சன் மண்டேலா எத்தனை ஆண்டுகள் சிறையில் இருந்...,27,494,tamil


In [34]:
test_df

,id,context,question,language
0,22bff3dec,"ज्वाला गुट्टा (जन्म: 7 सितंबर 1983; वर्धा, महा...",ज्वाला गुट्टा की माँ का नाम क्या है,hindi
1,282758170,गूगल मानचित्र (Google Maps) (पूर्व में गूगल लो...,गूगल मैप्स कब लॉन्च किया गया था?,hindi
2,d60987e0e,गुस्ताव रॉबर्ट किरचॉफ़ (१२ मार्च १८२४ - १७ अक्...,गुस्ताव किरचॉफ का जन्म कब हुआ था?,hindi
3,f99c770dc,அலுமினியம் (ஆங்கிலம்: அலுமினியம்; வட அமெரிக்க ...,அலுமினியத்தின் அணு எண் என்ன?,tamil
4,40dec1964,"கூட்டுறவு இயக்க வரலாறு, இங்கிலாந்து நாட்டில் ...",இந்தியாவில் பசுமை புரட்சியின் தந்தை என்று கருத...,tamil


In [70]:
def get_question_and_context_answer(example):
    dic = {}
    dic['question'] = example['question']
    dic['context'] = example['context'].strip() # remove any trailing and leading white spaces
    dic['answer'] = example['answer_text']
    start_idx= example['answer_start'] # find the starting index of the answer
    dic['start_idx']=start_idx
    dic['end_idx']= start_idx + len(example['answer_text'])
    return dic

In [74]:
# import transformer
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [87]:
max_length=384 # maximum length of question and answer
doc_stride=128
def tokenize(example):
    encoding = tokenizer(example['context'], example['question'],
                         max_length=max_length,
                         truncation="only_first",
                         return_overflowing_tokens=True,
                         stride=doc_stride)
    
    start_positions = encoding.char_to_token(example['start_idx'])
    end_positions = encoding.char_to_token(example['end_idx']-1)

    if start_positions is None:
        start_positions = max_length
    if end_positions is None:
        end_positions = max_length

    return {'input_ids': encoding['input_ids'],
          'attention_mask': encoding['attention_mask'],
          'start_positions': start_positions,
          'end_positions': end_positions}

In [88]:
# define the jaccard function
def jaccard(str1,str2):
  intersect=set(str1).intersection(set(str2))
  union=set(str1).union(set(str2))

  return len(intersect)/len(union)


In [89]:
import tensorflow as tf
from transformers import AutoModelForQuestionAnswering

In [90]:
model=AutoModelForQuestionAnswering.from_pretrained('distilbert-base-uncased')

epochs=8
kf=KFold(n_splits=10,shuffle=True,random_state=42)
loss_fn1 = tf.keras.losses.SparseCategoricalCrossentropy( from_logits=True)
loss_fn2 = tf.keras.losses.SparseCategoricalCrossentropy( from_logits=True)
opt = tf.keras.optimizers.Adam(learning_rate=3e-5)
losses=[]
valid_score=[]
val=0

for epoch in range(epochs):

  print("Starting epoch: %d"% epoch )
  val_score=[]

  for train_idx,valid_idx in kf.split(train_df):
    print("Valid: ",val)
    # train
    train=train_df.loc[train_idx,]

    # valid
    valid=train_df.loc[valid_idx,]

    # convert train and valid to transfromer dataset format
    train=d.Dataset.from_pandas(train)
    valid=d.Dataset.from_pandas(valid)

    # get question and features
    train= train.map(get_question_and_context_answer)
    valid=valid.map( get_question_and_context_answer)

    train=train.map(tokenize)
    valid=valid.map(tokenize)

    columns_to_return = ['input_ids','attention_mask', 'start_positions', 'end_positions']

    train.set_format(type='tf', columns=columns_to_return)
    valid.set_format(type='tf',columns=columns_to_return)


    train_features = {x: train[x].to_tensor(default_value=0, shape=[None, max_length]) for x in ['input_ids', 'attention_mask']}
    train_labels = {"start_positions": tf.reshape(train['start_positions'], shape=[-1,1]),
                'end_positions': tf.reshape(train['end_positions'], shape=[-1,1])}

    valid_features = {x: valid[x].to_tensor(default_value=0, shape=[None, max_length]) for x in ['input_ids', 'attention_mask']}
    valid_labels = {"start_positions": tf.reshape(valid['start_positions'], shape=[-1,1]),
                'end_positions': tf.reshape(valid['end_positions'], shape=[-1,1])}


    train_tfdataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).batch(16)

    for step, (x_batch_train, y_batch_train) in enumerate(train_tfdataset):
        with tf.GradientTape() as tape:
            answer_start_scores, answer_end_scores = model(x_batch_train)
            loss_start = loss_fn1(y_batch_train['start_positions'], answer_start_scores)
            loss_end = loss_fn2(y_batch_train['end_positions'], answer_end_scores)
            loss = 0.5 * (loss_start + loss_end)
        losses.append(loss)
        grads = tape.gradient(loss, model.trainable_weights)
        opt.apply_gradients(zip(grads, model.trainable_weights))

        if step % 5 == 0:
            print("Training loss (for one batch) at step %d: %.4f"% (step, 
                                                                    float(loss_start)))


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForQuestionAnswering: ['vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this mode

Starting epoch: 0
Valid:  0


  0%|          | 0/1002 [00:00<?, ?ex/s]

  0%|          | 0/112 [00:00<?, ?ex/s]

  0%|          | 0/1002 [00:00<?, ?ex/s]

  0%|          | 0/112 [00:00<?, ?ex/s]

/usr/local/lib/python3.7/dist-packages/datasets/formatting/formatting.py:167: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array(array, copy=False, **self.np_array_kwargs)


InvalidArgumentError: ignored

In [80]:
tr=d.Dataset.from_pandas(train_df)


In [81]:
tr.map( get_question_and_context_answer)

  0%|          | 0/1114 [00:00<?, ?ex/s]

Dataset({
    features: ['id', 'context', 'question', 'answer_text', 'answer_start', 'language', 'answer', 'start_idx', 'end_idx'],
    num_rows: 1114
})